In [1]:
# read block assignment file and store in dictionary:
#    assignment = { block_geoid : district_number }
def read_block_assignment_file(filepath, filename):
    assignment = dict()
    import pandas
    csvFile = pandas.read_csv( filepath + filename, skipinitialspace = True )
    for index, row in csvFile.iterrows():
        g = str( row[0] ) #str( row['GEOID'] ) #str( row['GEOID20'] )
        if len(g) < 15: # fix issue with leading zeros
            g = '0' + g
        j = str( row[1] ) # str( row['District'] )
        if j in { 'ZZ', 'ZZZ' }:
            print("Warning: geoid =",g,"is unassigned.")
            continue
        assignment[g] = int(j)
    return assignment

In [2]:
filepath = ''
filename = 'DE_SS.csv'
block_assignment = read_block_assignment_file(filepath, filename)
print("Number of blocks assigned:",len(block_assignment))

Number of blocks assigned: 20198


In [3]:
# read block-level graph
from gerrychain import Graph

filepath = ''
filename = 'DE_block.json'
G = Graph.from_json(filepath+filename)
for node in G.nodes:
    G.nodes[node]['TOTPOP'] = G.nodes[node]['P0010001']

In [4]:
import math

k = 21 #  number of districts
deviation = 0.075 # for this instance, use +/- 7.5% so that enacted plan is actually feasible
ideal_population = sum( G.nodes[i]['TOTPOP'] for i in G.nodes ) / k

L = math.ceil( (1-deviation) * ideal_population)
U = math.floor( (1+deviation) * ideal_population)
print("Using k, L, U =",k,L,U)

Using k, L, U = 21 43605 50675


In [5]:
import networkx as nx

def check_plan(G, L, U, k, plan):
    assert len(plan) == k
    for district in plan:
        assert nx.is_connected( G.subgraph(district) )
        population = sum( G.nodes[i]['TOTPOP'] for i in district )
        assert L <= population and population <= U
    return

In [6]:
node_to_district = { i : block_assignment[G.nodes[i]['GEOID20']] for i in G.nodes }

reference_plan = [ list() for j in range(k) ]
for i in node_to_district.keys():
    j = node_to_district[i] - 1
    reference_plan[j].append(i)
    
check_plan(G, L, U, k, reference_plan)

In [7]:
# Finds a minimal vertex subset that separates component from vertex b in digraph DG.
#
# This is algorithm 1 from Fischetti, Matteo, et al. 
#   "Thinning out Steiner trees: a node-based model for uniform edge costs." 
#   Mathematical Programming Computation 9.2 (2017): 203-229.
#
def find_minimal_separator(DG, component, b):
    neighbors_component = { i : False for i in DG.nodes }
    for i in nx.node_boundary(DG, component, None):
        neighbors_component[i] = True
    
    visited = { i : False for i in DG.nodes }
    child = [b]
    visited[b] = True
    
    while child:
        parent = child
        child = list()
        for i in parent:
            if not neighbors_component[i]:
                for j in DG.neighbors(i):
                    if not visited[j]:
                        child.append(j)
                        visited[j] = True
    
    C = [ i for i in DG.nodes if neighbors_component[i] and visited[i] ]
    return C

In [8]:
import gurobipy as gp
from gurobipy import GRB, LinExpr

def labeling_contiguity_callback(m, where):

    # check if LP relaxation at this branch-and-bound node has an integer solution
    if where != GRB.Callback.MIPSOL: 
        return
        
    # retrieve the LP solution
    xval = m.cbGetSolution(m._x)
    DG = m._DG
    AG = m._AG

    # check if each district is connected
    for j in range(m._k):

        # which nodes are selected?
        S = list( { i for i in AG.predecessors(j) if xval[i,j] > 0.5 } )
        if len(S)==1 or nx.is_strongly_connected( DG.subgraph(S) ):
            continue

        # for smaller connected components, find some vertex 'a' and add cut.
        max_comp = None
        
        for comp in sorted(nx.strongly_connected_components( DG.subgraph(S) ), key=len, reverse=True):
            if max_comp == None:
                max_comp = comp
                b = nx.utils.arbitrary_element( comp )
                continue

            # pick some vertex from this component
            a = nx.utils.arbitrary_element( comp )

            # add a,b-separator inequality
            C = find_minimal_separator(DG, comp, b)
            m.cbLazy( m._x[a,j] + m._x[b,j] <= 1 + gp.quicksum( m._x[c,j] for c in C if c in AG.predecessors(j)) )
            #m.cbLazy( m._x[a,j] + m._x[b,j] <= 1 + gp.quicksum( m._x[c,j] for c in C ) )
            #print("add a,b-separator inequality for a,b,C =",a,b,C)

In [9]:
# dist[i] = the distance from node i to the set of nodes S
# dist[i]=0 if i in S
# dist[i]=1 if i in N(S)
# dist[i]=2 if i in N^2(S)
# ...
# dist[i]=None if no path from i to S
#
def distance_to_vertex_set(G, S, cutoff=None):
    if cutoff==None:
        cutoff = G.number_of_nodes()
    
    dist = dict()
    touched = { i : False for i in G.nodes }
    child = S
    for s in S:
        touched[s] = True
    
    for h in range(1+cutoff):
        parent = child
        child = list()
        for p in parent:
            dist[p] = h
            for n in G.neighbors(p):
                if not touched[n]:
                    child.append(n)
                    touched[n] = True
    return dist

In [10]:
# G = input graph
# L = lower population limit
# U = upper population limit
# k = # of districts
# h = # hops permitted
def best_neighboring_plan(G, L, U, k, reference_plan, h=1):
    
    # for implementation convenience, we assume that G.nodes = {0, 1, ..., n-1}
    n = G.number_of_nodes()
    nodes = list(G.nodes)
    assert nodes == list( range(n) )
    
    # create a directed "assignment graph" AG 
    #  that has nodes {0, 1, ..., n-1}
    #  and edge (i,j) if node i can be assigned to district j in {0, 1, ..., k-1}
    AG = nx.DiGraph()
    AG.add_nodes_from( nodes )
    
    # add edges to AG
    for j in range(k):
        district = reference_plan[j]
        dist = distance_to_vertex_set(G, district, cutoff=h)
        AG.add_edges_from( [ (i,j) for i in dist.keys() ] )
        
    # initialize MIP
    m = gp.Model()
    x = m.addVars(AG.edges, vtype=GRB.BINARY)

    # assignment constraints
    m.addConstrs( gp.quicksum( x[i,j] for j in AG.neighbors(i) ) == 1 for i in G.nodes )

    # population balance
    m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in AG.predecessors(j) ) >= L for j in range(k) )
    m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in AG.predecessors(j) ) <= U for j in range(k) )

    # fix nodes that have only one choice
    num_nodes_fixed = 0
    for i in G.nodes:
        neigh = list(AG.neighbors(i)) 
        if len(neigh)==1:
            j = neigh[0]
            x[i,j].LB = 1
            num_nodes_fixed += 1
    m.update()

    print("this many nodes fixed:",num_nodes_fixed)
    print("out of a total:",G.number_of_nodes())
    print("remaining nodes:",G.number_of_nodes()-num_nodes_fixed)
    
    # which undirected edges (u,v) are cut because u->j but v!->j
    triples = list()
    for u,v in G.edges:
        for j in range(k):
            if j in AG.neighbors(u): # possible for u->j
                for j1 in AG.neighbors(v):
                    if j1 != j:  # possible for v!->j
                        triples.append((u,v,j))
                        break

    # cut edge constraints
    print("this many triples (y cut edge vars) =",len(triples))
    y = m.addVars(triples, vtype=GRB.BINARY)

    for u,v,j in triples:
        if j in AG.neighbors(v):
            m.addConstr( x[u,j] - x[v,j] <= y[u,v,j] )
        else:
            m.addConstr( x[u,j] == y[u,v,j] )
            
    # cut edges objective
    edges = { (u,v) for u,v,j in triples }
    print("this many pairs (is_cut edge vars) =",len(edges))
    is_cut = m.addVars( edges, vtype=GRB.BINARY)

    expr = { (u,v) : LinExpr() for u,v in edges }
    for u,v,j in triples:
        expr[u,v] += y[u,v,j]

    m.addConstrs( expr[u,v] == is_cut[u,v] for u,v in edges )
    
    # minimize # cut edges 
    m.setObjective( gp.quicksum( is_cut[u,v] for u,v in edges), GRB.MINIMIZE)
    
    # add contiguity constraints via callback
    m.Params.LazyConstraints = 1
    m._x = x
    m._k = k
    m._DG = nx.DiGraph(G)
    m._AG = AG
    m._callback = labeling_contiguity_callback
    
    # inject warm start (the reference plan itself)
    for j in range(k):
        for i in reference_plan[j]:
            m._x[i,j].start = 1
    
    # solve MIP 
    m.Params.IntFeasTol = 1e-7
    m.Params.FeasibilityTol = 1e-7
    m.optimize(m._callback)
    
    # double check that plan is feasible
    plan = [ [ i for i in AG.predecessors(j) if m._x[i,j].x > 0.5 ] for j in range(k) ]
    check_plan(G, L, U, k, plan)
    return plan

In [11]:
def mip_based_local_search(G, L, U, k, reference_plan, h=1, max_iterations=100):
    new_plan = reference_plan
    for iteration in range(max_iterations):
        old_plan = new_plan
        print("\nIteration",iteration,"of local search")
        new_plan = best_neighboring_plan(G, L, U, k, old_plan, h=1)
        if new_plan == old_plan:
            break
    return new_plan

In [ ]:
# 1-hop local search
plan = mip_based_local_search(G, L, U, k, reference_plan, h=1, max_iterations=100)


Iteration 0 of local search
Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-11
this many nodes fixed: 18293
out of a total: 20198
remaining nodes: 1905
this many triples (y cut edge vars) = 12077
this many pairs (is_cut edge vars) = 8368
Set parameter LazyConstraints to value 1
Set parameter IntFeasTol to value 1e-07
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-4790 CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 40685 rows, 42610 columns and 109285 nonzeros
Model fingerprint: 0x1641d141
Variable types: 0 continuous, 42610 integer (42610 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]

User MIP start produced solution with objective 1531 (0.58s

Loaded user MIP start with objective 1244
Processed MIP start in 1.06 seconds (0.02 work units)

Presolve removed 27231 rows and 27219 columns
Presolve time: 0.08s
Presolved: 10696 rows, 12315 columns, 32474 nonzeros
Variable types: 0 continuous, 12315 integer (12313 binary)

Root relaxation: objective 1.210000e+03, 5828 iterations, 0.04 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1210.00000    0    - 1244.00000 1210.00000  2.73%     -    2s
     0     0 1234.16667    0  160 1244.00000 1234.16667  0.79%     -    2s
     0     0 1238.00000    0    - 1244.00000 1238.00000  0.48%     -    3s
     0     0 1238.00000    0   41 1244.00000 1238.00000  0.48%     -    3s
     0     0 1238.00000    0    - 1244.00000 1238.00000  0.48%     -    4s
     0     0 1238.00000    0   16 1244.00000 1238.00000  0.48%     -    4s
     0     0 1238.00000    0    

In [ ]:
import csv

def export_to_baf(geoid_assignment, outfilename):
    with open(outfilename, 'w') as csvfile: 
        csvwriter = csv.writer(csvfile) 
        for geoid in geoid_assignment.keys():
            row = [geoid, geoid_assignment[geoid]+1]
            csvwriter.writerow(row) 
    return

In [ ]:
ga = { G.nodes[i]['GEOID20'] : j for j in range(k) for i in plan[j] }

export_to_baf(geoid_assignment=ga, outfilename='local_plan.csv')